In [25]:
from xgboost.sklearn import XGBClassifier
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np

In [26]:
train_values = pd.read_csv('train_values.csv', index_col='building_id')
train_values1 = pd.read_csv('train_values_short1.csv', index_col='building_id')
train_labels = pd.read_csv('train_labels.csv', index_col='building_id')
train_values_baseline = pd.get_dummies(train_values)
ambos_geos = train_values1.merge(train_values[["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"]],
                                            left_index=True, right_index=True)

In [ ]:
test  = pd.read_csv('test_values.csv', index_col='building_id')
test_baseline = pd.get_dummies(test)
test1 = pd.read_csv("Archivos auxiliares KNN/test_values_short1.csv", index_col='building_id')
ambos_geos_t = test1.merge(test[["geo_level_1_id", "geo_level_2_id", "geo_level_3_id"]],
                                            left_index=True, right_index=True)

In [ ]:
    xgb_short1 = XGBClassifier( objective = "multi:softmax", use_label_encoder=False, seed=30,
                         max_depth=9,
                         min_child_weight=6,
                         gamma=0.2554138025988315,
                        colsample_bytree = 0.7773754946860542,
                        subsample = 0.9030471581301206,
                         learning_rate =0.1, 
                         n_estimators=273, 
                         )    

In [30]:
xgb_baseline = XGBClassifier(random_state=2021, max_depth= 10, n_estimators= 273, verbosity=0)

In [31]:
    le = LabelEncoder()
    y = le.fit_transform(y=train_labels.values.ravel())


In [32]:
xgb_baseline.fit(train_values_baseline, y)

/home/guido/.local/lib/python3.6/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=273, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=2021, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=0)

In [37]:
preds = xgb_baseline.predict(test_baseline)
submission_format = pd.read_csv('submission_format.csv', index_col='building_id')
submission_horrible = pd.DataFrame(data=preds,
                             columns=submission_format.columns,
                             index=test.index)

In [38]:
submission_mejorada = submission_horrible+1

In [39]:
submission_mejorada.head()

,damage_grade
building_id,
300051,3
99355,2
890251,2
745817,1
421793,3


In [40]:
submission_mejorada.to_csv('subs/xgboost_baseline_n_estimators273.csv', index=True)

In [42]:
probas = xgb_baseline.predict_proba(test_baseline)

In [43]:
ids = test.index

In [45]:
df_probas = pd.DataFrame(data=probas,
                             columns=[1,2,3],
                             index=ids)

In [46]:
df_probas.to_csv('predict_probas/xgboost_baseline.csv', index=True)